In [1]:
from ortools.linear_solver import pywraplp

In [2]:
M = 1000

In [3]:
def even_constraint(last_state, next_state_var, solver):
    n = solver.IntVar(0, solver.infinity(), 'n_even')
    b = solver.BoolVar('b_even')
    solver.Add(next_state_var - 2 * n <= M * (1 - b))
    solver.Add(next_state_var - 2 * n >= -M * (1 - b))
    return [(b, 1)]

def odd_constraint(last_state, next_state_var, solver):
    n = solver.IntVar(0, solver.infinity(), 'n_odd')
    b = solver.BoolVar('b_odd')
    solver.Add(next_state_var - 2 * n - 1 <= M * (1 - b))
    solver.Add(next_state_var - 2 * n - 1 >= -M * (1 - b))
    return [(b, 1)]

def two_changes_constraint(last_state, next_state_var, solver):
    b_2_greater = solver.BoolVar('b_2_greater')
    b_1_greater = solver.BoolVar('b_1_greater')
    b_no_change = solver.BoolVar('b_no_change')
    solver.Add(next_state_var - (last_state + 2) <= M * (1 - b_2_greater))
    solver.Add(next_state_var - (last_state + 2) >= -M * (1 - b_2_greater))
    solver.Add(next_state_var - (last_state + 1) <= M * (1 - b_1_greater))
    solver.Add(next_state_var - (last_state + 1) >= -M * (1 - b_1_greater))
    solver.Add(next_state_var - last_state <= M * (1 - b_no_change))
    solver.Add(next_state_var - last_state >= -M * (1 - b_no_change))
    return [(b_2_greater, 1), (b_1_greater, 0.5), (b_no_change, 0)]

def one_more_or_less_constraint(last_state, next_state_var, solver):
    b1 = solver.BoolVar('b_one_more')
    b2 = solver.BoolVar('b_one_less')
    solver.Add(next_state_var - (last_state + 1) <= M * (1 - b1))
    solver.Add(next_state_var - (last_state + 1) >= -M * (1 - b1))
    solver.Add(next_state_var - (last_state - 1) <= M * (1 - b2))
    solver.Add(next_state_var - (last_state - 1) >= -M * (1 - b2))
    return [(b1, 1), (b2, 1)]

In [10]:
def machine(state, constraints_dict):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        raise Exception('SCIP solver not available.')
        
    solver.SetTimeLimit(30000)
    
    next_state_var = solver.IntVar(0.0, solver.infinity(), 'next_state_var')
    
    objective = solver.Objective()
    objective.SetMaximization()
    bool_vars = {} # Dictionary to store solver boolean variables
    
    for name, (constraint_func, weight) in constraints_dict.items():
        for b, w in constraint_func(state, next_state_var, solver):
            bool_vars[b] = name # Store the constraint name
            objective.SetCoefficient(b, w * weight)
    
    status = solver.Solve()
    total_objective = objective.Value() # Get the total objective value
    satisfied_constraints = []
    
    if status in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE, pywraplp.Solver.ABNORMAL]:
        new_state = next_state_var.solution_value()
    
        for b in bool_vars.keys():
            if b.solution_value() > 0.5:
                satisfied_constraints.append(bool_vars[b]) # Get the constraint name
        return new_state, satisfied_constraints, total_objective
    else:
        return state, [], total_objective

In [11]:
constraints_dict = {
    'even': (even_constraint, 1),
    'odd': (odd_constraint, 1),
    'two_changes': (two_changes_constraint, 1),
    'one_more_or_less': (one_more_or_less_constraint, 1)
}

In [12]:
new_state, satisfied, total_objective = machine(5.0, constraints_dict)
print(f"New state: {new_state}, Satisfied constraints: {satisfied}, Total Objective: {total_objective}")

New state: 6.0, Satisfied constraints: ['even', 'two_changes', 'one_more_or_less'], Total Objective: 2.4999999999999996


In [13]:
def run_machine_in_sequence(initial_state, list_of_constraints_dicts):
    current_state = initial_state
    output_states = []
    all_satisfied_constraints = []
    
    for constraints_dict in list_of_constraints_dicts:
        new_state, satisfied_constraints, total_objective = machine(current_state, constraints_dict)
        output_states.append(new_state)
        all_satisfied_constraints.append((satisfied_constraints, total_objective))
        current_state = new_state # Update the current state for the next iteration
        
    return output_states, all_satisfied_constraints

In [14]:
run_machine_in_sequence(0, [constraints_dict for x in range(10)] )

([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 [(['odd', 'two_changes', 'one_more_or_less'], 2.5),
  (['even', 'two_changes', 'one_more_or_less'], 2.5),
  (['odd', 'two_changes', 'one_more_or_less'], 2.5),
  (['even', 'two_changes', 'one_more_or_less'], 2.5),
  (['odd', 'two_changes', 'one_more_or_less'], 2.4999999999999996),
  (['even', 'two_changes', 'one_more_or_less'], 2.4999999999999996),
  (['odd', 'two_changes', 'one_more_or_less'], 2.5),
  (['even', 'two_changes', 'one_more_or_less'], 2.5),
  (['odd', 'two_changes', 'one_more_or_less'], 2.4999999999999996),
  (['even', 'two_changes', 'one_more_or_less'], 2.4999999999999996)])